In [1]:
from hepmc import *
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
def write_events(E_CM, sample, filename):
    n_out = int(sample.data.shape[1]/4)
    
    with open(filename, "w") as f:
        f.write("\nHepMC::Version 2.06.09\nHepMC::IO_GenEvent-START_EVENT_LISTING\n")
        for i in range(sample.size):
            # event
            # E evt_number no_mpi scale alphq_qcd alpha_qed signal_id barcode_signal_process_vertex no_vertices barcode_particle_1 barcode_particle_2 no_random_state {random_state} no_weights {weights}
            f.write("E %i -1 0 1.0000000000000000e+00 1.0000000000000000e+00 0 0 1 10001 10002 0 1 %e\n" % (i, sample.weights[i]))
            
            # weights
            f.write("N 1 \"0\"\n")
            
            # units
            f.write("U GEV MM\n")
            
            # vertex
            # V barcode id x y z ctau no_incoming no_outgoing no_weights {weights}
            f.write("V -1 0 0 0 0 0 2 %i 0\n" % n_out)
            
            # incoming particles
            # P barcode PDG_id px py pz energy gen_mass status_code pol_theta pol_phi barcode_vertex_incoming no_flow {code_index, code}
            f.write("P 10001 11 0 0 %e %e 0 4 0 0 -1 0\n" % (E_CM/2, E_CM/2))
            f.write("P 10002 -11 0 0 %e %e 0 4 0 0 -1 0\n" % (-E_CM/2, E_CM/2))
            
            # outgoing particles
            for j in range(n_out):
                if j == 0:
                    pid = 1
                elif j == 1:
                    pid = -1
                else:
                    pid = 21
                
                E = sample.data[i, 4*j]
                px = sample.data[i, 4*j+1]
                py = sample.data[i, 4*j+2]
                pz = sample.data[i, 4*j+3]
                f.write("P %i %i %e %e %e %e 0 1 0 0 0 0\n" % (10003+j, pid, px, py, pz, E))
            
        f.write("HepMC::IO_GenEvent-END_EVENT_LISTING")

In [3]:
sarge = densities.Sarge(2, 3, 100.)

In [4]:
eeqqg = densities.ee_qq_1g(100., 5., .3)

In [5]:
rambo_mapping = phase_space.RamboOnDiet(100., 3)
mapped = phase_space.MappedDensity(eeqqg, rambo_mapping)

In [21]:
%%time
sarge_sample = ImportanceMC(sarge)(eeqqg, 10000)
sarge_sample.integral, sarge_sample.integral_err

CPU times: user 8.65 s, sys: 9.16 ms, total: 8.66 s
Wall time: 8.72 s


In [22]:
ys = eeqqg.pdf(sarge_sample.data)
#ratio = ys*sarge_sample.weights
sarge_sample.weights = ys/sarge_sample.weights
write_events(100., sarge_sample, "../samples/qcd/2-3/sarge_weighted.hepmc")

In [23]:
bound = sarge_sample.weights.max()

In [24]:
%%time
# acceptance-rejection sampler
sarge_sample_unweighted = np.empty((10000, 12))
for i in range(10000):
    aprob = 0
    while np.random.random() > aprob:
        proposal = sarge.proposal()
        f = eeqqg.pdf(proposal)
        g = sarge.proposal_pdf(None, proposal)
        aprob = f/(bound*g)
        if aprob > 1:
            print("bound too low")
    sarge_sample_unweighted[i] = proposal
sarge_sample_unweighted = Sample(data=sarge_sample_unweighted, weights=np.full(10000, 1./10000))

bound too low
bound too low
bound too low
bound too low
bound too low
bound too low
bound too low
bound too low
CPU times: user 4min 16s, sys: 119 ms, total: 4min 16s
Wall time: 4min 18s


In [25]:
write_events(100., sarge_sample_unweighted, "../samples/qcd/2-3/sarge_unweighted.hepmc")

In [10]:
rambo_sample = PlainMC(5)(mapped, 10000)
rambo_sample.integral, rambo_sample.integral_err

(111.80528304517088, 3.540252954412419)

In [11]:
ys = mapped.pdf(rambo_sample.data)
max_y = ys.max()

In [12]:
%%time
rambo = AcceptRejectSampler(mapped, max_y, 5)
rambo_sample = rambo.sample(1000)

CPU times: user 44.1 s, sys: 0 ns, total: 44.1 s
Wall time: 44.1 s


In [13]:
rambo_sample.data = rambo_mapping.map(rambo_sample.data)
rambo_sample.weights = np.full(rambo_sample.size, 1./rambo_sample.size)

In [14]:
write_events(100., rambo_sample, "../samples/qcd/2-3/rambo.hepmc")

In [15]:
# just a local sampler (rambo)
met = DefaultMetropolis(5, mapped, proposals.Gaussian(5, .01))
%time met_sample = met.sample(100000, np.random.rand(5))

Generated 5000 samples.
Generated 10000 samples.
Generated 15000 samples.
Generated 20000 samples.
Generated 25000 samples.
Generated 30000 samples.
Generated 35000 samples.
Generated 40000 samples.
Generated 45000 samples.
Generated 50000 samples.
Generated 55000 samples.
Generated 60000 samples.
Generated 65000 samples.
Generated 70000 samples.
Generated 75000 samples.
Generated 80000 samples.
Generated 85000 samples.
Generated 90000 samples.
Generated 95000 samples.
Generated 100000 samples.
CPU times: user 2min 4s, sys: 389 µs, total: 2min 4s
Wall time: 2min 5s


In [16]:
met_sample.data = rambo_mapping.map(met_sample.data)
met_sample.weights = np.full(met_sample.size, 1./met_sample.size)
write_events(100., met_sample, "../samples/qcd/2-3/metropolis.hepmc")

In [17]:
# now sarge + local in MC3
importance = DefaultMetropolis(3 * 4, eeqqg, proposal=sarge)  # using sarge, full momentum sample space
local = DefaultMetropolis(5, mapped, proposals.Gaussian(5, .01))  # using rambo, rambo sample space
update = MixingMarkovUpdate(5, [importance, local], target=mapped,
                            out_maps={0: rambo_mapping.map_inverse},
                            in_maps={0: rambo_mapping.map}) # rambo sample space, important to set target explicitly
%time mc3_sample = update.sample(100000, np.random.rand(5))

Generated 5000 samples.
Generated 10000 samples.
Generated 15000 samples.
Generated 20000 samples.
Generated 25000 samples.
Generated 30000 samples.
Generated 35000 samples.
Generated 40000 samples.
Generated 45000 samples.
Generated 50000 samples.
Generated 55000 samples.
Generated 60000 samples.
Generated 65000 samples.
Generated 70000 samples.
Generated 75000 samples.
Generated 80000 samples.
Generated 85000 samples.
Generated 90000 samples.
Generated 95000 samples.
Generated 100000 samples.
CPU times: user 3min 19s, sys: 0 ns, total: 3min 19s
Wall time: 3min 20s


In [18]:
mc3_sample.data = rambo_mapping.map(mc3_sample.data)
mc3_sample.weights = np.full(mc3_sample.size, 1./mc3_sample.size)
write_events(100., mc3_sample, "../samples/qcd/2-3/mc3.hepmc")